In [ ]:
# Required libraries
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def full_from_bottomright_quadrant(q5: np.ndarray) -> np.ndarray:
    """
    Build a 10×10 unit cell from a 5×5 bottom-right quadrant.
    Mirrors:
      TL = flipud(fliplr(q5))
      TR = flipud(q5)
      BL = fliplr(q5)
      BR = q5
    """
    BR = q5
    BL = np.fliplr(q5)
    TR = np.flipud(q5)
    TL = np.flipud(np.fliplr(q5))
    top = np.hstack([TL, TR])
    bottom = np.hstack([BL, BR])
    return np.vstack([top, bottom])

def generate_from_csv_bottomright_v2(
    csv_path: str,
    out_dir: str,
    *,
    skip_header_like_first_row: bool = True,
    max_rows: int | None = None,
    show_first: int = 0,
    order: str = "C",
    transpose_quadrant: bool = False,
    dpi: int = 220,
    start_index: int = 0,
    basename: str = "unitcell_row_",
):
    csv_path = Path(csv_path)
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)

    try:
        df = pd.read_csv(csv_path, header=None)
    except Exception:
        df = pd.read_csv(csv_path, header=None, sep=";", engine="python")

    if skip_header_like_first_row:
        # Heuristic: if any of columns 26..29 are non-numeric in the first row, drop it
        # (0-based indexing: cols 26..29 are the 27th..30th columns)
        if df.shape[1] >= 30:
            first_row_tail = df.iloc[0, 26:30].astype(str).str.contains(r"[A-Za-z_]", regex=True).any()
            if first_row_tail:
                df = df.iloc[1:].reset_index(drop=True)

    # Must have at least 25 columns for the 5×5 quadrant
    if df.shape[1] < 25:
        raise ValueError(f"Need at least 25 columns, found {df.shape[1]} in {csv_path.name}.")

    # Coerce to numeric for the first 25 columns; fill NaNs with 0 and cast to int (0/1)
    bin_part = df.iloc[:, :25].apply(pd.to_numeric, errors="coerce").fillna(0).astype(int)
    df.iloc[:, :25] = bin_part

    # Colormap: 0 -> yellow (void), 1 -> black (material)
    cmap = ListedColormap(["yellow", "black"])

    total_rows = len(df)
    n_rows = total_rows if max_rows is None else min(max_rows, total_rows)
    print(f"[INFO] Processing {n_rows}/{total_rows} rows from {csv_path.name}.")

    for idx in range(n_rows):
        row_vals = df.iloc[idx, :25].to_numpy(dtype=int)

        # Rebuild 5×5 bottom-right quadrant
        q5 = row_vals.reshape(5, 5, order=order)
        if transpose_quadrant:
            q5 = q5.T

        full = full_from_bottomright_quadrant(q5)

        # Save PNG
        fig = plt.figure(figsize=(3.6, 3.6))
        plt.imshow(full, cmap=cmap, origin="upper", interpolation="none", vmin=0, vmax=1)
        plt.xticks(range(10)); plt.yticks(range(10))
        plt.grid(True, linestyle="--", linewidth=0.3, alpha=0.6)
        plt.title(f"Unit Cell (csv_row {idx})")
        plt.tight_layout()

        file_idx = start_index + idx
        out_file = out_dir / f"{basename}{file_idx:04d}.png"
        plt.savefig(out_file, dpi=dpi)
        plt.close(fig)

        # Optional previews
        if idx < show_first:
            plt.figure(figsize=(3.6, 3.6))
            plt.imshow(full, cmap=cmap, origin="upper", interpolation="none", vmin=0, vmax=1)
            plt.xticks(range(10)); plt.yticks(range(10))
            plt.grid(True, linestyle="--", linewidth=0.3, alpha=0.6)
            plt.title(f"Preview — csv_row {idx} -> file_idx {file_idx:04d}")
            plt.show()

    print(f"[DONE] Saved images in: {out_dir.resolve()}")


In [ ]:
generate_from_csv_bottomright_v2(
    csv_path=r"G:\Other computers\Dell G3\Universitat\2nd Semester\MLMM\New\corrected_data_10k.csv",
    out_dir=r"G:\Other computers\Dell G3\Universitat\2nd Semester\MLMM\New\Images_10k",
    skip_header_like_first_row=True,
    max_rows=None,
    show_first=3,
    order="C",
    transpose_quadrant=False,
    dpi=220,
    start_index=1,
    basename="unitcell_row_",
)
